In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [9]:
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score

--- GradientBoosting ---

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)

selector= RFE(estimator= grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
cm
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      8517
           1       0.40      0.25      0.31       912

    accuracy                           0.89      9429
   macro avg       0.66      0.61      0.62      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.39522998296422485
Accuracy: 0.8902322621699014
AUC: 0.8212241667336123


--- Tuned GradientBoosting ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

tuned_grb= gs_grb.best_estimator_

selector= RFE(estimator= tuned_grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
cm
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.44      0.13      0.21       912

    accuracy                           0.90      9429
   macro avg       0.68      0.56      0.58      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.4404332129963899
Accuracy: 0.8997772828507795
AUC: 0.819407173784526


--- Tuned GradientBoosting 1---

In [12]:
param_grid= {
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 8],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.6, 0.8, 1.0]
}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

tuned_grb1= gs_grb.best_estimator_

selector= RFE(estimator= tuned_grb1, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb1.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb1= tuned_grb1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb1= tuned_grb1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb1, labels= tuned_grb1.classes_)
cm
print(classification_report(y_test, y_pred_tuned_grb1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb1))

Fitting 5 folds for each of 1215 candidates, totalling 6075 fits
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.52      0.07      0.12       912

    accuracy                           0.90      9429
   macro avg       0.72      0.53      0.53      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.5217391304347826
Accuracy: 0.903807402693817
AUC: 0.8194450881518696


--- Tuned GradientBoosting RandomizedSearch 2 ---

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_dist = {
    'learning_rate': uniform(0.01, 0.2),
    'n_estimators': randint(100, 300),
    'max_depth': randint(3, 8),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4),
    'subsample': uniform(0.6, 0.4)
}

rs_grb= RandomizedSearchCV(estimator= grb, param_distributions= param_dist, n_iter= 200, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2, random_state= 15)
rs_grb.fit(X_sm_tl, y_sm_tl)

tuned_grb2= rs_grb.best_estimator_

selector= RFE(estimator= tuned_grb2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb2= tuned_grb2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb2= tuned_grb2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb2, labels= tuned_grb2.classes_)
cm
print(classification_report(y_test, y_pred_tuned_grb2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb2))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.48      0.07      0.12       912

    accuracy                           0.90      9429
   macro avg       0.70      0.53      0.53      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.484375
Accuracy: 0.9028529006257291
AUC: 0.8182436726134935


--- XGB ---

In [10]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)

selector= RFE(estimator= xgb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
cm
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
selected_feature_names= X_sm_tl_df.columns[selected_features]
print("Selected Features:", selected_feature_names)

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.46      0.10      0.16       912

    accuracy                           0.90      9429
   macro avg       0.68      0.54      0.55      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.45789473684210524
Accuracy: 0.901580231201612
AUC: 0.804015807394499
Selected Features: Index(['GenHlth', 'HighBP_1', 'HighChol_1', 'Smoker_1', 'Diabetes_1',
       'PhysActivity_1', 'Fruits_1', 'Veggies_1', 'DiffWalk_1', 'Sex_1'],
      dtype='object')


--- Tuned XGB ---

In [89]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

tuned_xgb= gs_xgb.best_estimator_

selector= RFE(estimator= tuned_xgb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
cm
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9687437596374279
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9687437596374279
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.42      0.13      0.20       912

    accuracy                           0.90      9429
   macro avg       0.67      0.56      0.57      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.42142857142857143
Accuracy: 0.8986106692120055
AUC: 0.816064079271797
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.42      0.13      0.20       912

    accuracy                           0.90      9429
   macro avg       0.67      0.56      0.57      9429
weighted avg       0.87      0.90   

--- Tuned XGB RandomizedSearch 1---

In [11]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

rs_xgb= RandomizedSearchCV(estimator=xgb, param_distributions= param_dist, n_iter= 200, scoring= 'precision', cv= 5, random_state=15, n_jobs= -1, verbose= 2)
rs_xgb.fit(X_sm_tl, y_sm_tl)

tuned_xgb1= rs_xgb.best_estimator_

selector= RFE(estimator= tuned_xgb1, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_xgb1.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb1= tuned_xgb1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb1= tuned_xgb1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb1, labels= tuned_xgb1.classes_)
cm
print(classification_report(y_test, y_pred_tuned_xgb1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb1))

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
selected_feature_names= X_sm_tl_df.columns[selected_features]
print("Selected Features:", selected_feature_names)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.51      0.08      0.14       912

    accuracy                           0.90      9429
   macro avg       0.71      0.53      0.54      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.5144927536231884
Accuracy: 0.9037013469084738
AUC: 0.8205988693407817


--- Tuned XGB RandomizedSearch 2---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'learning_rate': [0.03, 0.05, 0.07],
    'n_estimators': [250, 300, 350],
    'max_depth': [6, 7, 8],
    'min_child_weight': [4, 5, 6],
    'gamma': [0.05, 0.1, 0.15],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'scale_pos_weight': [1, 3, 5]
}


gs_xgb = GridSearchCV(estimator=tuned_xgb1, param_grid=param_grid, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

tuned_xgb2= gs_xgb.best_estimator_

selector= RFE(estimator= tuned_xgb2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_xgb2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb2= tuned_xgb2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb2= tuned_xgb2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb2, labels= tuned_xgb2.classes_)
cm
print(classification_report(y_test, y_pred_tuned_xgb2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb2))

Fitting 5 folds for each of 6561 candidates, totalling 32805 fits
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.48      0.11      0.17       912

    accuracy                           0.90      9429
   macro avg       0.69      0.55      0.56      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.47549019607843135
Accuracy: 0.9022165659136706
AUC: 0.8209202724581796


--- LGBM ---

In [10]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)

selector= RFE(estimator= lgm, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
cm
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008896 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009939 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8569
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.0

[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6202
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5947
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.0

--- Tuned LGBM ---

In [91]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

tuned_lgm= gs_lgm.best_estimator_

selector= RFE(estimator= tuned_lgm, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
cm
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007884 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the tr

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8443
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7393
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6712
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003755 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5182
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006336 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002607 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 

--- Tuned LGBM RandomizedSearch 1---

In [11]:
from sklearn.model_selection import RandomizedSearchCV

param_grid= {
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'num_leaves': [15, 31, 63, 127],
    'min_data_in_leaf': [20, 30, 50, 100, 200],
    'max_depth': [-1, 5, 10, 20],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_split_gain': [0, 0.1, 0.2],
    'scale_pos_weight': [1, 2, 5, 10]
}

rs_lgm= RandomizedSearchCV(estimator= lgm, param_distributions= param_grid, n_iter= 200, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2, random_state= 15)
rs_lgm.fit(X_sm_tl, y_sm_tl)
tuned_lgm1= rs_lgm.best_estimator_

selector= RFE(estimator= tuned_lgm1, n_features_to_select= 10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_lgm1.fit(X_sm_tl_selected, y_sm_tl, eval_set= [(X_sm_tl_test_selected, y_test)], eval_metric= 'logloss')

y_pred_tuned_lgm1= tuned_lgm1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm1= tuned_lgm1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm1, labels= tuned_lgm1.classes_)
cm
print(classification_report(y_test, y_pred_tuned_lgm1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm1))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011581 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 42
[LightGB

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> i

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008500 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8569
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006660 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8500
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006704 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8476
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 10
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.

--- Tuned LGBM 2---

In [13]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param_grid_fine= {
    'learning_rate': [0.03, 0.05, 0.07],
    'num_leaves': [31, 63],
    'min_data_in_leaf': [50, 100],
    'max_depth': [10, 15],
    'reg_alpha': [0.3, 0.5],
    'reg_lambda': [0.3, 0.5],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.7, 0.8],
    'min_split_gain': [0.1, 0.2],
    'scale_pos_weight': [5, 10, 20]  
}

gs_lgm= GridSearchCV(estimator= tuned_lgm1, param_grid= param_grid_fine, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

tuned_lgm2= gs_lgm.best_estimator_

selector= RFE(estimator= tuned_lgm2, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_lgm2.fit(X_sm_tl_selected, y_sm_tl, eval_set= [(X_sm_tl_test_selected, y_test)], eval_metric= 'logloss')

y_pred_tuned_lgm2= tuned_lgm2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm2= tuned_lgm2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm2, labels= tuned_lgm2.classes_)
cm
print(classification_report(y_test, y_pred_tuned_lgm2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm2))

Fitting 5 folds for each of 2304 candidates, totalling 11520 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022498 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightG

[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005869 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7583
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004956 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[L

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
              precision    recall  f1-score   support

           0       0.94      0.82      0.88      8517
           1       0.25      0.55      0.35       912

    accuracy                           0.80      9429
   macro avg       0.60      0.69      0.61      9429
weighted avg       0.88      0.80      0.83      9429

Precision: 0.2511233150274588
Accuracy: 0.7975395057800403
AUC: 0.7698836717689492


--- CatBoost ---

In [10]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)

selector= RFE(estimator= cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
cm
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.062428
0:	learn: 0.6250216	total: 209ms	remaining: 3m 28s
1:	learn: 0.5692396	total: 238ms	remaining: 1m 58s
2:	learn: 0.5267455	total: 266ms	remaining: 1m 28s
3:	learn: 0.4884263	total: 288ms	remaining: 1m 11s
4:	learn: 0.4548822	total: 307ms	remaining: 1m 1s
5:	learn: 0.4285898	total: 323ms	remaining: 53.5s
6:	learn: 0.4089582	total: 340ms	remaining: 48.3s
7:	learn: 0.3917242	total: 356ms	remaining: 44.1s
8:	learn: 0.3741989	total: 371ms	remaining: 40.8s
9:	learn: 0.3594983	total: 387ms	remaining: 38.3s
10:	learn: 0.3465981	total: 402ms	remaining: 36.1s
11:	learn: 0.3328883	total: 422ms	remaining: 34.7s
12:	learn: 0.3227633	total: 439ms	remaining: 33.3s
13:	learn: 0.3174098	total: 453ms	remaining: 31.9s
14:	learn: 0.3102871	total: 467ms	remaining: 30.7s
15:	learn: 0.3024371	total: 482ms	remaining: 29.6s
16:	learn: 0.2912392	total: 497ms	remaining: 28.7s
17:	learn: 0.2850467	total: 511ms	remaining: 27.9s
18:	learn: 0.2749619	total: 526ms	remaining: 27.2s
19:	lea

161:	learn: 0.1556985	total: 2.9s	remaining: 15s
162:	learn: 0.1550086	total: 2.91s	remaining: 15s
163:	learn: 0.1549262	total: 2.93s	remaining: 14.9s
164:	learn: 0.1545799	total: 2.94s	remaining: 14.9s
165:	learn: 0.1545041	total: 2.96s	remaining: 14.9s
166:	learn: 0.1543881	total: 2.97s	remaining: 14.8s
167:	learn: 0.1541451	total: 2.99s	remaining: 14.8s
168:	learn: 0.1540455	total: 3.01s	remaining: 14.8s
169:	learn: 0.1539882	total: 3.02s	remaining: 14.8s
170:	learn: 0.1538743	total: 3.04s	remaining: 14.7s
171:	learn: 0.1537996	total: 3.06s	remaining: 14.7s
172:	learn: 0.1537200	total: 3.07s	remaining: 14.7s
173:	learn: 0.1529509	total: 3.09s	remaining: 14.7s
174:	learn: 0.1526168	total: 3.11s	remaining: 14.7s
175:	learn: 0.1524008	total: 3.13s	remaining: 14.6s
176:	learn: 0.1520775	total: 3.14s	remaining: 14.6s
177:	learn: 0.1520248	total: 3.16s	remaining: 14.6s
178:	learn: 0.1519408	total: 3.17s	remaining: 14.5s
179:	learn: 0.1516430	total: 3.18s	remaining: 14.5s
180:	learn: 0.151

333:	learn: 0.1347230	total: 5.75s	remaining: 11.5s
334:	learn: 0.1345956	total: 5.76s	remaining: 11.4s
335:	learn: 0.1345232	total: 5.78s	remaining: 11.4s
336:	learn: 0.1344796	total: 5.79s	remaining: 11.4s
337:	learn: 0.1344383	total: 5.8s	remaining: 11.4s
338:	learn: 0.1344086	total: 5.82s	remaining: 11.3s
339:	learn: 0.1343399	total: 5.83s	remaining: 11.3s
340:	learn: 0.1342627	total: 5.85s	remaining: 11.3s
341:	learn: 0.1342024	total: 5.86s	remaining: 11.3s
342:	learn: 0.1341653	total: 5.87s	remaining: 11.3s
343:	learn: 0.1341163	total: 5.89s	remaining: 11.2s
344:	learn: 0.1340466	total: 5.9s	remaining: 11.2s
345:	learn: 0.1340027	total: 5.92s	remaining: 11.2s
346:	learn: 0.1339521	total: 5.93s	remaining: 11.2s
347:	learn: 0.1338773	total: 5.95s	remaining: 11.1s
348:	learn: 0.1338356	total: 5.96s	remaining: 11.1s
349:	learn: 0.1337581	total: 5.98s	remaining: 11.1s
350:	learn: 0.1336442	total: 5.99s	remaining: 11.1s
351:	learn: 0.1335873	total: 6.01s	remaining: 11.1s
352:	learn: 0.

500:	learn: 0.1253383	total: 8.62s	remaining: 8.58s
501:	learn: 0.1253024	total: 8.64s	remaining: 8.57s
502:	learn: 0.1252645	total: 8.65s	remaining: 8.55s
503:	learn: 0.1252081	total: 8.67s	remaining: 8.53s
504:	learn: 0.1251726	total: 8.69s	remaining: 8.52s
505:	learn: 0.1251034	total: 8.7s	remaining: 8.5s
506:	learn: 0.1250742	total: 8.72s	remaining: 8.48s
507:	learn: 0.1250237	total: 8.74s	remaining: 8.46s
508:	learn: 0.1249763	total: 8.75s	remaining: 8.44s
509:	learn: 0.1249354	total: 8.77s	remaining: 8.43s
510:	learn: 0.1248622	total: 8.79s	remaining: 8.41s
511:	learn: 0.1248250	total: 8.81s	remaining: 8.39s
512:	learn: 0.1247621	total: 8.83s	remaining: 8.38s
513:	learn: 0.1247075	total: 8.85s	remaining: 8.36s
514:	learn: 0.1246712	total: 8.87s	remaining: 8.35s
515:	learn: 0.1246333	total: 8.88s	remaining: 8.33s
516:	learn: 0.1245891	total: 8.91s	remaining: 8.32s
517:	learn: 0.1245579	total: 8.93s	remaining: 8.31s
518:	learn: 0.1245227	total: 8.95s	remaining: 8.29s
519:	learn: 0.

667:	learn: 0.1182422	total: 11.5s	remaining: 5.7s
668:	learn: 0.1182070	total: 11.5s	remaining: 5.69s
669:	learn: 0.1181806	total: 11.5s	remaining: 5.67s
670:	learn: 0.1181282	total: 11.5s	remaining: 5.65s
671:	learn: 0.1180528	total: 11.5s	remaining: 5.64s
672:	learn: 0.1180109	total: 11.6s	remaining: 5.62s
673:	learn: 0.1179832	total: 11.6s	remaining: 5.6s
674:	learn: 0.1179601	total: 11.6s	remaining: 5.58s
675:	learn: 0.1179171	total: 11.6s	remaining: 5.56s
676:	learn: 0.1178868	total: 11.6s	remaining: 5.54s
677:	learn: 0.1178126	total: 11.6s	remaining: 5.53s
678:	learn: 0.1177852	total: 11.6s	remaining: 5.51s
679:	learn: 0.1177304	total: 11.7s	remaining: 5.49s
680:	learn: 0.1176979	total: 11.7s	remaining: 5.47s
681:	learn: 0.1176669	total: 11.7s	remaining: 5.45s
682:	learn: 0.1176437	total: 11.7s	remaining: 5.43s
683:	learn: 0.1176205	total: 11.7s	remaining: 5.42s
684:	learn: 0.1175933	total: 11.7s	remaining: 5.4s
685:	learn: 0.1175547	total: 11.8s	remaining: 5.38s
686:	learn: 0.1

...

242:	learn: 0.1543377	total: 2.98s	remaining: 9.28s
243:	learn: 0.1542616	total: 2.99s	remaining: 9.27s
244:	learn: 0.1542359	total: 3s	remaining: 9.26s
245:	learn: 0.1541368	total: 3.01s	remaining: 9.24s
246:	learn: 0.1539936	total: 3.03s	remaining: 9.23s
247:	learn: 0.1539687	total: 3.04s	remaining: 9.21s
248:	learn: 0.1539031	total: 3.05s	remaining: 9.2s
249:	learn: 0.1536426	total: 3.06s	remaining: 9.19s
250:	learn: 0.1533481	total: 3.08s	remaining: 9.18s
251:	learn: 0.1533166	total: 3.09s	remaining: 9.16s
252:	learn: 0.1532344	total: 3.1s	remaining: 9.15s
253:	learn: 0.1531574	total: 3.11s	remaining: 9.14s
254:	learn: 0.1528867	total: 3.13s	remaining: 9.13s
255:	learn: 0.1528193	total: 3.14s	remaining: 9.13s
256:	learn: 0.1526185	total: 3.16s	remaining: 9.14s
257:	learn: 0.1525757	total: 3.18s	remaining: 9.14s
258:	learn: 0.1525267	total: 3.19s	remaining: 9.14s
259:	learn: 0.1524078	total: 3.21s	remaining: 9.13s
260:	learn: 0.1523486	total: 3.22s	remaining: 9.13s
261:	learn: 0.152

409:	learn: 0.1453450	total: 5.07s	remaining: 7.3s
410:	learn: 0.1453247	total: 5.08s	remaining: 7.29s
411:	learn: 0.1452994	total: 5.1s	remaining: 7.28s
412:	learn: 0.1452734	total: 5.11s	remaining: 7.26s
413:	learn: 0.1452568	total: 5.12s	remaining: 7.25s
414:	learn: 0.1452324	total: 5.13s	remaining: 7.24s
415:	learn: 0.1452005	total: 5.15s	remaining: 7.22s
416:	learn: 0.1451698	total: 5.16s	remaining: 7.21s
417:	learn: 0.1451231	total: 5.17s	remaining: 7.2s
418:	learn: 0.1451015	total: 5.18s	remaining: 7.19s
419:	learn: 0.1450731	total: 5.19s	remaining: 7.17s
420:	learn: 0.1450567	total: 5.21s	remaining: 7.16s
421:	learn: 0.1450298	total: 5.22s	remaining: 7.15s
422:	learn: 0.1449983	total: 5.23s	remaining: 7.13s
423:	learn: 0.1449746	total: 5.24s	remaining: 7.12s
424:	learn: 0.1449472	total: 5.26s	remaining: 7.11s
425:	learn: 0.1449220	total: 5.27s	remaining: 7.1s
426:	learn: 0.1448539	total: 5.28s	remaining: 7.09s
427:	learn: 0.1448245	total: 5.29s	remaining: 7.08s
428:	learn: 0.14

575:	learn: 0.1408258	total: 7.22s	remaining: 5.31s
576:	learn: 0.1408077	total: 7.23s	remaining: 5.3s
577:	learn: 0.1407924	total: 7.25s	remaining: 5.29s
578:	learn: 0.1407690	total: 7.26s	remaining: 5.28s
579:	learn: 0.1407443	total: 7.28s	remaining: 5.27s
580:	learn: 0.1407289	total: 7.29s	remaining: 5.26s
581:	learn: 0.1407052	total: 7.31s	remaining: 5.25s
582:	learn: 0.1406823	total: 7.33s	remaining: 5.24s
583:	learn: 0.1406528	total: 7.35s	remaining: 5.23s
584:	learn: 0.1406346	total: 7.36s	remaining: 5.22s
585:	learn: 0.1406214	total: 7.38s	remaining: 5.21s
586:	learn: 0.1405970	total: 7.4s	remaining: 5.2s
587:	learn: 0.1405807	total: 7.41s	remaining: 5.19s
588:	learn: 0.1405597	total: 7.43s	remaining: 5.18s
589:	learn: 0.1405314	total: 7.44s	remaining: 5.17s
590:	learn: 0.1405060	total: 7.46s	remaining: 5.16s
591:	learn: 0.1404837	total: 7.47s	remaining: 5.15s
592:	learn: 0.1404601	total: 7.48s	remaining: 5.13s
593:	learn: 0.1404519	total: 7.49s	remaining: 5.12s
594:	learn: 0.1

736:	learn: 0.1371630	total: 9.36s	remaining: 3.34s
737:	learn: 0.1371413	total: 9.37s	remaining: 3.33s
738:	learn: 0.1371244	total: 9.38s	remaining: 3.31s
739:	learn: 0.1371030	total: 9.4s	remaining: 3.3s
740:	learn: 0.1370886	total: 9.41s	remaining: 3.29s
741:	learn: 0.1370632	total: 9.43s	remaining: 3.28s
742:	learn: 0.1370469	total: 9.45s	remaining: 3.27s
743:	learn: 0.1370223	total: 9.46s	remaining: 3.25s
744:	learn: 0.1370018	total: 9.48s	remaining: 3.24s
745:	learn: 0.1369799	total: 9.49s	remaining: 3.23s
746:	learn: 0.1369721	total: 9.51s	remaining: 3.22s
747:	learn: 0.1369535	total: 9.52s	remaining: 3.21s
748:	learn: 0.1369404	total: 9.53s	remaining: 3.19s
749:	learn: 0.1369288	total: 9.54s	remaining: 3.18s
750:	learn: 0.1368879	total: 9.55s	remaining: 3.17s
751:	learn: 0.1368737	total: 9.56s	remaining: 3.15s
752:	learn: 0.1368504	total: 9.57s	remaining: 3.14s
753:	learn: 0.1368359	total: 9.58s	remaining: 3.13s
754:	learn: 0.1368143	total: 9.59s	remaining: 3.11s
755:	learn: 0.

902:	learn: 0.1341108	total: 11.5s	remaining: 1.24s
903:	learn: 0.1340875	total: 11.5s	remaining: 1.22s
904:	learn: 0.1340712	total: 11.5s	remaining: 1.21s
905:	learn: 0.1340488	total: 11.6s	remaining: 1.2s
906:	learn: 0.1340368	total: 11.6s	remaining: 1.19s
907:	learn: 0.1340267	total: 11.6s	remaining: 1.17s
908:	learn: 0.1339992	total: 11.6s	remaining: 1.16s
909:	learn: 0.1339864	total: 11.6s	remaining: 1.15s
910:	learn: 0.1339752	total: 11.6s	remaining: 1.14s
911:	learn: 0.1339623	total: 11.6s	remaining: 1.12s
912:	learn: 0.1339537	total: 11.6s	remaining: 1.11s
913:	learn: 0.1339336	total: 11.7s	remaining: 1.1s
914:	learn: 0.1339264	total: 11.7s	remaining: 1.08s
915:	learn: 0.1339164	total: 11.7s	remaining: 1.07s
916:	learn: 0.1339119	total: 11.7s	remaining: 1.06s
917:	learn: 0.1338897	total: 11.7s	remaining: 1.05s
918:	learn: 0.1338654	total: 11.7s	remaining: 1.03s
919:	learn: 0.1338464	total: 11.7s	remaining: 1.02s
920:	learn: 0.1338345	total: 11.7s	remaining: 1.01s
921:	learn: 0.

--- Tuned CatBoost ---

In [93]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

tuned_cat= gs_cat.best_estimator_

selector= RFE(estimator= tuned_cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
cm
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4923625	total: 14.1ms	remaining: 692ms
1:	learn: 0.4102545	total: 30.6ms	remaining: 734ms
2:	learn: 0.3467355	total: 49.6ms	remaining: 776ms
3:	learn: 0.2992550	total: 68.1ms	remaining: 783ms
4:	learn: 0.2826226	total: 82.8ms	remaining: 745ms
5:	learn: 0.2584881	total: 104ms	remaining: 762ms
6:	learn: 0.2322352	total: 119ms	remaining: 730ms
7:	learn: 0.2252604	total: 133ms	remaining: 699ms
8:	learn: 0.2194321	total: 147ms	remaining: 671ms
9:	learn: 0.2123800	total: 162ms	remaining: 647ms
10:	learn: 0.1999490	total: 180ms	remaining: 638ms
11:	learn: 0.1915375	total: 198ms	remaining: 627ms
0:	learn: 0.4923625	total: 14.1ms	remaining: 692ms
1:	learn: 0.4102545	total: 30.6ms	remaining: 734ms
2:	learn: 0.3467355	total: 49.6ms	remaining: 776ms
3:	learn: 0.2992550	total: 68.1ms	remaining: 783ms
4:	learn: 0.2826226	total: 82.8ms	remaining: 745ms
5:	learn: 0.25

30:	learn: 0.1490579	total: 422ms	remaining: 259ms
31:	learn: 0.1484102	total: 433ms	remaining: 243ms
32:	learn: 0.1476666	total: 445ms	remaining: 229ms
33:	learn: 0.1463616	total: 459ms	remaining: 216ms
34:	learn: 0.1462107	total: 474ms	remaining: 203ms
35:	learn: 0.1460656	total: 488ms	remaining: 190ms
36:	learn: 0.1458348	total: 503ms	remaining: 177ms
37:	learn: 0.1445214	total: 518ms	remaining: 164ms
38:	learn: 0.1442947	total: 530ms	remaining: 149ms
39:	learn: 0.1440775	total: 544ms	remaining: 136ms
40:	learn: 0.1435968	total: 556ms	remaining: 122ms
41:	learn: 0.1435303	total: 568ms	remaining: 108ms
42:	learn: 0.1431038	total: 581ms	remaining: 94.6ms
43:	learn: 0.1429679	total: 592ms	remaining: 80.8ms
44:	learn: 0.1427063	total: 604ms	remaining: 67.1ms
30:	learn: 0.1490579	total: 422ms	remaining: 259ms
31:	learn: 0.1484102	total: 433ms	remaining: 243ms
32:	learn: 0.1476666	total: 445ms	remaining: 229ms
33:	learn: 0.1463616	total: 459ms	remaining: 216ms
34:	learn: 0.1462107	total: 

16:	learn: 0.1705126	total: 210ms	remaining: 407ms
17:	learn: 0.1647320	total: 221ms	remaining: 394ms
18:	learn: 0.1634533	total: 233ms	remaining: 380ms
19:	learn: 0.1581059	total: 244ms	remaining: 366ms
20:	learn: 0.1575463	total: 255ms	remaining: 353ms
21:	learn: 0.1567177	total: 266ms	remaining: 338ms
22:	learn: 0.1563917	total: 279ms	remaining: 327ms
23:	learn: 0.1552909	total: 289ms	remaining: 314ms
24:	learn: 0.1530390	total: 300ms	remaining: 300ms
25:	learn: 0.1509681	total: 313ms	remaining: 289ms
26:	learn: 0.1506446	total: 323ms	remaining: 275ms
27:	learn: 0.1503245	total: 335ms	remaining: 263ms
28:	learn: 0.1489292	total: 347ms	remaining: 251ms
29:	learn: 0.1486714	total: 358ms	remaining: 239ms
30:	learn: 0.1485214	total: 369ms	remaining: 226ms
31:	learn: 0.1481735	total: 379ms	remaining: 213ms
32:	learn: 0.1477364	total: 391ms	remaining: 201ms
16:	learn: 0.1705126	total: 210ms	remaining: 407ms
17:	learn: 0.1647320	total: 221ms	remaining: 394ms
18:	learn: 0.1634533	total: 233

49:	learn: 0.1427972	total: 601ms	remaining: 0us
0:	learn: 0.4821317	total: 15.6ms	remaining: 763ms
1:	learn: 0.3957770	total: 29.5ms	remaining: 708ms
2:	learn: 0.3222090	total: 44ms	remaining: 689ms
49:	learn: 0.1427972	total: 601ms	remaining: 0us
0:	learn: 0.4821317	total: 15.6ms	remaining: 763ms
1:	learn: 0.3957770	total: 29.5ms	remaining: 708ms
2:	learn: 0.3222090	total: 44ms	remaining: 689ms
3:	learn: 0.2764971	total: 59.4ms	remaining: 684ms
4:	learn: 0.2643193	total: 69.7ms	remaining: 627ms
5:	learn: 0.2401075	total: 81.2ms	remaining: 596ms
6:	learn: 0.2328214	total: 91ms	remaining: 559ms
7:	learn: 0.2200712	total: 103ms	remaining: 539ms
8:	learn: 0.2154766	total: 113ms	remaining: 515ms
9:	learn: 0.2069499	total: 123ms	remaining: 494ms
10:	learn: 0.1964206	total: 134ms	remaining: 476ms
11:	learn: 0.1854377	total: 146ms	remaining: 461ms
12:	learn: 0.1773595	total: 157ms	remaining: 446ms
13:	learn: 0.1752377	total: 167ms	remaining: 431ms
14:	learn: 0.1733999	total: 178ms	remaining:

32:	learn: 0.1485643	total: 418ms	remaining: 215ms
33:	learn: 0.1469246	total: 430ms	remaining: 202ms
34:	learn: 0.1467580	total: 440ms	remaining: 189ms
35:	learn: 0.1466218	total: 450ms	remaining: 175ms
36:	learn: 0.1452725	total: 461ms	remaining: 162ms
37:	learn: 0.1450830	total: 470ms	remaining: 149ms
38:	learn: 0.1448089	total: 480ms	remaining: 136ms
39:	learn: 0.1446379	total: 491ms	remaining: 123ms
40:	learn: 0.1433640	total: 503ms	remaining: 110ms
41:	learn: 0.1431594	total: 513ms	remaining: 97.7ms
42:	learn: 0.1421189	total: 524ms	remaining: 85.4ms
43:	learn: 0.1419561	total: 534ms	remaining: 72.9ms
44:	learn: 0.1416579	total: 544ms	remaining: 60.4ms
45:	learn: 0.1403507	total: 556ms	remaining: 48.4ms
46:	learn: 0.1402423	total: 565ms	remaining: 36.1ms
47:	learn: 0.1401097	total: 576ms	remaining: 24ms
48:	learn: 0.1399906	total: 587ms	remaining: 12ms
49:	learn: 0.1390086	total: 597ms	remaining: 0us
32:	learn: 0.1485643	total: 418ms	remaining: 215ms
33:	learn: 0.1469246	total: 4

...

4:	learn: 0.3095585	total: 42.2ms	remaining: 380ms
5:	learn: 0.2903811	total: 49.2ms	remaining: 361ms
6:	learn: 0.2722538	total: 56.4ms	remaining: 346ms
7:	learn: 0.2576462	total: 63.7ms	remaining: 334ms
8:	learn: 0.2503478	total: 69.6ms	remaining: 317ms
9:	learn: 0.2357970	total: 76.5ms	remaining: 306ms
10:	learn: 0.2249824	total: 82.4ms	remaining: 292ms
11:	learn: 0.2167382	total: 88.4ms	remaining: 280ms
12:	learn: 0.2088138	total: 94.3ms	remaining: 268ms
13:	learn: 0.2009580	total: 99.6ms	remaining: 256ms
14:	learn: 0.1912810	total: 105ms	remaining: 245ms
15:	learn: 0.1896983	total: 111ms	remaining: 235ms
16:	learn: 0.1885233	total: 116ms	remaining: 225ms
17:	learn: 0.1859918	total: 121ms	remaining: 215ms
18:	learn: 0.1836511	total: 126ms	remaining: 206ms
19:	learn: 0.1830077	total: 132ms	remaining: 197ms
20:	learn: 0.1805430	total: 137ms	remaining: 189ms
21:	learn: 0.1782197	total: 142ms	remaining: 181ms
22:	learn: 0.1762340	total: 147ms	remaining: 173ms
23:	learn: 0.1757885	total:

38:	learn: 0.1594100	total: 215ms	remaining: 60.5ms
39:	learn: 0.1579443	total: 220ms	remaining: 54.9ms
40:	learn: 0.1576300	total: 224ms	remaining: 49.3ms
41:	learn: 0.1567902	total: 229ms	remaining: 43.7ms
42:	learn: 0.1565950	total: 234ms	remaining: 38.1ms
43:	learn: 0.1564320	total: 239ms	remaining: 32.5ms
44:	learn: 0.1557055	total: 244ms	remaining: 27.1ms
45:	learn: 0.1553144	total: 249ms	remaining: 21.7ms
46:	learn: 0.1541978	total: 254ms	remaining: 16.2ms
47:	learn: 0.1530205	total: 261ms	remaining: 10.9ms
48:	learn: 0.1525523	total: 267ms	remaining: 5.45ms
49:	learn: 0.1523181	total: 273ms	remaining: 0us
0:	learn: 0.4967656	total: 7.35ms	remaining: 360ms
1:	learn: 0.4195564	total: 13.9ms	remaining: 334ms
2:	learn: 0.3723401	total: 19.9ms	remaining: 311ms
3:	learn: 0.3363526	total: 27.5ms	remaining: 316ms
4:	learn: 0.3034832	total: 32.1ms	remaining: 289ms
38:	learn: 0.1594100	total: 215ms	remaining: 60.5ms
39:	learn: 0.1579443	total: 220ms	remaining: 54.9ms
40:	learn: 0.1576300

15:	learn: 0.2002517	total: 81.7ms	remaining: 174ms
16:	learn: 0.1963056	total: 87.8ms	remaining: 170ms
17:	learn: 0.1954078	total: 93ms	remaining: 165ms
18:	learn: 0.1944089	total: 98.1ms	remaining: 160ms
19:	learn: 0.1923055	total: 103ms	remaining: 155ms
20:	learn: 0.1900305	total: 108ms	remaining: 150ms
21:	learn: 0.1876461	total: 114ms	remaining: 145ms
22:	learn: 0.1868282	total: 120ms	remaining: 140ms
23:	learn: 0.1843179	total: 125ms	remaining: 135ms
24:	learn: 0.1821787	total: 130ms	remaining: 130ms
25:	learn: 0.1803496	total: 136ms	remaining: 125ms
26:	learn: 0.1787690	total: 141ms	remaining: 120ms
27:	learn: 0.1778862	total: 147ms	remaining: 115ms
28:	learn: 0.1760072	total: 152ms	remaining: 110ms
29:	learn: 0.1747529	total: 157ms	remaining: 105ms
30:	learn: 0.1734508	total: 162ms	remaining: 99.6ms
31:	learn: 0.1731961	total: 167ms	remaining: 94.2ms
32:	learn: 0.1722652	total: 172ms	remaining: 88.5ms
33:	learn: 0.1710426	total: 177ms	remaining: 83.1ms
34:	learn: 0.1707425	tota

10:	learn: 0.2302253	total: 62.9ms	remaining: 223ms
11:	learn: 0.2204694	total: 68.1ms	remaining: 216ms
12:	learn: 0.2108309	total: 74.3ms	remaining: 211ms
13:	learn: 0.2039533	total: 79.3ms	remaining: 204ms
14:	learn: 0.2027474	total: 84.2ms	remaining: 196ms
15:	learn: 0.2014288	total: 88.8ms	remaining: 189ms
16:	learn: 0.1979043	total: 93.3ms	remaining: 181ms
17:	learn: 0.1970432	total: 98ms	remaining: 174ms
18:	learn: 0.1964171	total: 102ms	remaining: 167ms
19:	learn: 0.1939345	total: 107ms	remaining: 161ms
20:	learn: 0.1916492	total: 112ms	remaining: 154ms
21:	learn: 0.1895874	total: 116ms	remaining: 148ms
22:	learn: 0.1887223	total: 121ms	remaining: 142ms
23:	learn: 0.1862679	total: 125ms	remaining: 136ms
24:	learn: 0.1843677	total: 130ms	remaining: 130ms
25:	learn: 0.1812185	total: 135ms	remaining: 124ms
26:	learn: 0.1798303	total: 139ms	remaining: 118ms
27:	learn: 0.1778183	total: 143ms	remaining: 113ms
28:	learn: 0.1754774	total: 148ms	remaining: 107ms
29:	learn: 0.1732239	tota

1:	learn: 0.4055472	total: 12.9ms	remaining: 309ms
2:	learn: 0.3556939	total: 19.6ms	remaining: 308ms
3:	learn: 0.3112961	total: 24.2ms	remaining: 278ms
4:	learn: 0.2883165	total: 28.7ms	remaining: 258ms
5:	learn: 0.2761855	total: 34.1ms	remaining: 250ms
6:	learn: 0.2679276	total: 38.5ms	remaining: 236ms
7:	learn: 0.2541808	total: 42.9ms	remaining: 225ms
8:	learn: 0.2417904	total: 48ms	remaining: 219ms
9:	learn: 0.2376267	total: 52.6ms	remaining: 210ms
10:	learn: 0.2287885	total: 57.1ms	remaining: 203ms
11:	learn: 0.2208120	total: 61.8ms	remaining: 196ms
12:	learn: 0.2189807	total: 66.6ms	remaining: 190ms
13:	learn: 0.2150532	total: 71.2ms	remaining: 183ms
14:	learn: 0.2116470	total: 75.9ms	remaining: 177ms
15:	learn: 0.2078798	total: 80.5ms	remaining: 171ms
16:	learn: 0.2062773	total: 84.8ms	remaining: 165ms
17:	learn: 0.2031010	total: 89.4ms	remaining: 159ms
18:	learn: 0.1980868	total: 94.1ms	remaining: 153ms
19:	learn: 0.1962775	total: 98.6ms	remaining: 148ms
20:	learn: 0.1930349	to

--- Tuned CatBoost RandomizedSearch 1---

In [12]:
from sklearn.model_selection import RandomizedSearchCV
param_dist= {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.005, 0.01, 0.02, 0.05, 0.1],
    'iterations': [100, 200, 500],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'border_count': [32, 64, 128],
    'bagging_temperature': [0.0, 0.5, 1.0],
    'scale_pos_weight': [1, 2, 5, 10],
    'random_strength': [1, 2, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bylevel': [0.8, 0.9, 1.0]
}

rs_cat= RandomizedSearchCV(estimator= cat, param_distributions= param_dist, n_iter= 100, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2, random_state= 15)
rs_cat.fit(X_sm_tl, y_sm_tl)

tuned_cat1= rs_cat.best_estimator_

selector= RFE(estimator= tuned_cat1, n_features_to_select= 10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat1.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_cat1= tuned_cat1.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat1= tuned_cat1.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat1, labels= tuned_cat1.classes_)
cm
print(classification_report(y_test, y_pred_tuned_cat1, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat1, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat1))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat1))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
0:	learn: 0.6029397	total: 21.8ms	remaining: 10.9s
1:	learn: 0.5271894	total: 41.2ms	remaining: 10.3s
2:	learn: 0.4871201	total: 66.1ms	remaining: 11s
3:	learn: 0.4365970	total: 87ms	remaining: 10.8s
4:	learn: 0.3974080	total: 106ms	remaining: 10.5s
5:	learn: 0.3723935	total: 129ms	remaining: 10.6s
6:	learn: 0.3526087	total: 152ms	remaining: 10.7s
7:	learn: 0.3384945	total: 176ms	remaining: 10.8s
8:	learn: 0.3295063	total: 198ms	remaining: 10.8s
9:	learn: 0.3174583	total: 221ms	remaining: 10.8s
10:	learn: 0.3025300	total: 242ms	remaining: 10.8s
11:	learn: 0.2917114	total: 266ms	remaining: 10.8s
12:	learn: 0.2772792	total: 289ms	remaining: 10.8s
13:	learn: 0.2687138	total: 315ms	remaining: 10.9s
14:	learn: 0.2634917	total: 338ms	remaining: 10.9s
15:	learn: 0.2569723	total: 357ms	remaining: 10.8s
16:	learn: 0.2545638	total: 381ms	remaining: 10.8s
17:	learn: 0.2453910	total: 401ms	remaining: 10.7s
18:	learn: 0.2380441	total: 4

165:	learn: 0.1410367	total: 3.32s	remaining: 6.68s
166:	learn: 0.1409371	total: 3.35s	remaining: 6.67s
167:	learn: 0.1408342	total: 3.37s	remaining: 6.66s
168:	learn: 0.1407266	total: 3.39s	remaining: 6.64s
169:	learn: 0.1406394	total: 3.42s	remaining: 6.63s
170:	learn: 0.1405683	total: 3.44s	remaining: 6.62s
171:	learn: 0.1404434	total: 3.47s	remaining: 6.61s
172:	learn: 0.1403561	total: 3.49s	remaining: 6.6s
173:	learn: 0.1402078	total: 3.52s	remaining: 6.59s
174:	learn: 0.1400855	total: 3.54s	remaining: 6.58s
175:	learn: 0.1400048	total: 3.57s	remaining: 6.57s
176:	learn: 0.1399030	total: 3.59s	remaining: 6.55s
177:	learn: 0.1397963	total: 3.62s	remaining: 6.54s
178:	learn: 0.1396919	total: 3.64s	remaining: 6.52s
179:	learn: 0.1393366	total: 3.66s	remaining: 6.51s
180:	learn: 0.1392235	total: 3.68s	remaining: 6.49s
181:	learn: 0.1390970	total: 3.7s	remaining: 6.46s
182:	learn: 0.1390066	total: 3.72s	remaining: 6.44s
183:	learn: 0.1389092	total: 3.74s	remaining: 6.43s
184:	learn: 0.

335:	learn: 0.1244120	total: 6.38s	remaining: 3.12s
336:	learn: 0.1243468	total: 6.4s	remaining: 3.09s
337:	learn: 0.1242971	total: 6.41s	remaining: 3.07s
338:	learn: 0.1242198	total: 6.42s	remaining: 3.05s
339:	learn: 0.1241585	total: 6.45s	remaining: 3.03s
340:	learn: 0.1240794	total: 6.46s	remaining: 3.01s
341:	learn: 0.1240226	total: 6.47s	remaining: 2.99s
342:	learn: 0.1239681	total: 6.49s	remaining: 2.97s
343:	learn: 0.1238422	total: 6.5s	remaining: 2.95s
344:	learn: 0.1237879	total: 6.51s	remaining: 2.93s
345:	learn: 0.1236925	total: 6.53s	remaining: 2.91s
346:	learn: 0.1236285	total: 6.54s	remaining: 2.88s
347:	learn: 0.1235249	total: 6.55s	remaining: 2.86s
348:	learn: 0.1234816	total: 6.57s	remaining: 2.84s
349:	learn: 0.1234050	total: 6.58s	remaining: 2.82s
350:	learn: 0.1233683	total: 6.59s	remaining: 2.8s
351:	learn: 0.1233099	total: 6.61s	remaining: 2.78s
352:	learn: 0.1232198	total: 6.62s	remaining: 2.76s
353:	learn: 0.1231542	total: 6.63s	remaining: 2.74s
354:	learn: 0.1

499:	learn: 0.1141860	total: 9.01s	remaining: 0us
0:	learn: 0.6029397	total: 31.6ms	remaining: 15.8s
1:	learn: 0.5271894	total: 59.8ms	remaining: 14.9s
2:	learn: 0.4871201	total: 82.8ms	remaining: 13.7s
3:	learn: 0.4365970	total: 102ms	remaining: 12.7s
4:	learn: 0.3974080	total: 116ms	remaining: 11.5s
5:	learn: 0.3723935	total: 129ms	remaining: 10.6s
6:	learn: 0.3526087	total: 142ms	remaining: 9.98s
7:	learn: 0.3384945	total: 155ms	remaining: 9.53s
8:	learn: 0.3295063	total: 167ms	remaining: 9.1s
9:	learn: 0.3174583	total: 180ms	remaining: 8.8s
10:	learn: 0.3025300	total: 192ms	remaining: 8.54s
11:	learn: 0.2917114	total: 205ms	remaining: 8.33s
12:	learn: 0.2772792	total: 218ms	remaining: 8.17s
13:	learn: 0.2687138	total: 230ms	remaining: 8s
14:	learn: 0.2634917	total: 245ms	remaining: 7.91s
15:	learn: 0.2569723	total: 262ms	remaining: 7.93s
16:	learn: 0.2545638	total: 276ms	remaining: 7.85s
17:	learn: 0.2453910	total: 289ms	remaining: 7.75s
18:	learn: 0.2380441	total: 302ms	remaining:

164:	learn: 0.1411296	total: 2.45s	remaining: 4.97s
165:	learn: 0.1410367	total: 2.47s	remaining: 4.96s
166:	learn: 0.1409371	total: 2.48s	remaining: 4.95s
167:	learn: 0.1408342	total: 2.5s	remaining: 4.94s
168:	learn: 0.1407266	total: 2.52s	remaining: 4.94s
169:	learn: 0.1406394	total: 2.54s	remaining: 4.93s
170:	learn: 0.1405683	total: 2.56s	remaining: 4.92s
171:	learn: 0.1404434	total: 2.57s	remaining: 4.91s
172:	learn: 0.1403561	total: 2.59s	remaining: 4.9s
173:	learn: 0.1402078	total: 2.6s	remaining: 4.88s
174:	learn: 0.1400855	total: 2.62s	remaining: 4.87s
175:	learn: 0.1400048	total: 2.64s	remaining: 4.86s
176:	learn: 0.1399030	total: 2.65s	remaining: 4.84s
177:	learn: 0.1397963	total: 2.67s	remaining: 4.83s
178:	learn: 0.1396919	total: 2.68s	remaining: 4.81s
179:	learn: 0.1393366	total: 2.7s	remaining: 4.8s
180:	learn: 0.1392235	total: 2.72s	remaining: 4.79s
181:	learn: 0.1390970	total: 2.73s	remaining: 4.77s
182:	learn: 0.1390066	total: 2.74s	remaining: 4.75s
183:	learn: 0.138

...

264:	learn: 0.1486281	total: 3.36s	remaining: 2.98s
265:	learn: 0.1485617	total: 3.37s	remaining: 2.97s
266:	learn: 0.1485159	total: 3.39s	remaining: 2.96s
267:	learn: 0.1484694	total: 3.4s	remaining: 2.94s
268:	learn: 0.1484132	total: 3.41s	remaining: 2.93s
269:	learn: 0.1483685	total: 3.42s	remaining: 2.92s
270:	learn: 0.1483334	total: 3.43s	remaining: 2.9s
271:	learn: 0.1482846	total: 3.44s	remaining: 2.89s
272:	learn: 0.1482451	total: 3.46s	remaining: 2.88s
273:	learn: 0.1481900	total: 3.47s	remaining: 2.86s
274:	learn: 0.1481465	total: 3.49s	remaining: 2.85s
275:	learn: 0.1481138	total: 3.5s	remaining: 2.84s
276:	learn: 0.1480703	total: 3.51s	remaining: 2.83s
277:	learn: 0.1479442	total: 3.52s	remaining: 2.81s
278:	learn: 0.1479142	total: 3.53s	remaining: 2.8s
279:	learn: 0.1478837	total: 3.54s	remaining: 2.78s
280:	learn: 0.1478530	total: 3.55s	remaining: 2.77s
281:	learn: 0.1477823	total: 3.56s	remaining: 2.75s
282:	learn: 0.1477553	total: 3.57s	remaining: 2.74s
283:	learn: 0.14

438:	learn: 0.1410058	total: 5.5s	remaining: 765ms
439:	learn: 0.1409829	total: 5.51s	remaining: 752ms
440:	learn: 0.1409518	total: 5.52s	remaining: 739ms
441:	learn: 0.1409179	total: 5.53s	remaining: 726ms
442:	learn: 0.1408869	total: 5.54s	remaining: 713ms
443:	learn: 0.1408621	total: 5.55s	remaining: 701ms
444:	learn: 0.1408124	total: 5.56s	remaining: 688ms
445:	learn: 0.1407918	total: 5.57s	remaining: 675ms
446:	learn: 0.1407691	total: 5.58s	remaining: 662ms
447:	learn: 0.1407275	total: 5.59s	remaining: 649ms
448:	learn: 0.1406922	total: 5.61s	remaining: 637ms
449:	learn: 0.1406645	total: 5.62s	remaining: 624ms
450:	learn: 0.1406342	total: 5.63s	remaining: 611ms
451:	learn: 0.1406157	total: 5.64s	remaining: 599ms
452:	learn: 0.1405792	total: 5.65s	remaining: 586ms
453:	learn: 0.1405573	total: 5.66s	remaining: 573ms
454:	learn: 0.1405156	total: 5.67s	remaining: 560ms
455:	learn: 0.1404955	total: 5.68s	remaining: 548ms
456:	learn: 0.1404666	total: 5.69s	remaining: 535ms
457:	learn: 0

98:	learn: 0.1730287	total: 1.19s	remaining: 4.84s
99:	learn: 0.1728856	total: 1.21s	remaining: 4.83s
100:	learn: 0.1724477	total: 1.22s	remaining: 4.81s
101:	learn: 0.1723768	total: 1.23s	remaining: 4.8s
102:	learn: 0.1719276	total: 1.24s	remaining: 4.79s
103:	learn: 0.1718333	total: 1.25s	remaining: 4.77s
104:	learn: 0.1717570	total: 1.26s	remaining: 4.75s
105:	learn: 0.1716646	total: 1.28s	remaining: 4.75s
106:	learn: 0.1712684	total: 1.29s	remaining: 4.75s
107:	learn: 0.1709609	total: 1.3s	remaining: 4.74s
108:	learn: 0.1705728	total: 1.32s	remaining: 4.73s
109:	learn: 0.1704664	total: 1.33s	remaining: 4.71s
110:	learn: 0.1701930	total: 1.34s	remaining: 4.7s
111:	learn: 0.1701008	total: 1.35s	remaining: 4.68s
112:	learn: 0.1698215	total: 1.36s	remaining: 4.68s
113:	learn: 0.1697249	total: 1.38s	remaining: 4.66s
114:	learn: 0.1696384	total: 1.39s	remaining: 4.65s
115:	learn: 0.1692690	total: 1.4s	remaining: 4.64s
116:	learn: 0.1687798	total: 1.41s	remaining: 4.63s
117:	learn: 0.1686

266:	learn: 0.1485159	total: 3.14s	remaining: 2.74s
267:	learn: 0.1484694	total: 3.15s	remaining: 2.73s
268:	learn: 0.1484132	total: 3.16s	remaining: 2.71s
269:	learn: 0.1483685	total: 3.17s	remaining: 2.7s
270:	learn: 0.1483334	total: 3.18s	remaining: 2.69s
271:	learn: 0.1482846	total: 3.19s	remaining: 2.68s
272:	learn: 0.1482451	total: 3.21s	remaining: 2.67s
273:	learn: 0.1481900	total: 3.22s	remaining: 2.65s
274:	learn: 0.1481465	total: 3.23s	remaining: 2.64s
275:	learn: 0.1481138	total: 3.24s	remaining: 2.63s
276:	learn: 0.1480703	total: 3.25s	remaining: 2.61s
277:	learn: 0.1479442	total: 3.26s	remaining: 2.6s
278:	learn: 0.1479142	total: 3.27s	remaining: 2.59s
279:	learn: 0.1478837	total: 3.28s	remaining: 2.58s
280:	learn: 0.1478530	total: 3.3s	remaining: 2.57s
281:	learn: 0.1477823	total: 3.31s	remaining: 2.56s
282:	learn: 0.1477553	total: 3.32s	remaining: 2.55s
283:	learn: 0.1477218	total: 3.34s	remaining: 2.54s
284:	learn: 0.1476837	total: 3.36s	remaining: 2.54s
285:	learn: 0.1

433:	learn: 0.1411877	total: 5.3s	remaining: 806ms
434:	learn: 0.1411328	total: 5.31s	remaining: 793ms
435:	learn: 0.1411010	total: 5.32s	remaining: 781ms
436:	learn: 0.1410628	total: 5.33s	remaining: 768ms
437:	learn: 0.1410364	total: 5.34s	remaining: 756ms
438:	learn: 0.1410058	total: 5.35s	remaining: 744ms
439:	learn: 0.1409829	total: 5.36s	remaining: 731ms
440:	learn: 0.1409518	total: 5.37s	remaining: 719ms
441:	learn: 0.1409179	total: 5.38s	remaining: 706ms
442:	learn: 0.1408869	total: 5.39s	remaining: 694ms
443:	learn: 0.1408621	total: 5.4s	remaining: 682ms
444:	learn: 0.1408124	total: 5.41s	remaining: 669ms
445:	learn: 0.1407918	total: 5.42s	remaining: 657ms
446:	learn: 0.1407691	total: 5.43s	remaining: 644ms
447:	learn: 0.1407275	total: 5.44s	remaining: 632ms
448:	learn: 0.1406922	total: 5.45s	remaining: 620ms
449:	learn: 0.1406645	total: 5.46s	remaining: 607ms
450:	learn: 0.1406342	total: 5.48s	remaining: 595ms
451:	learn: 0.1406157	total: 5.49s	remaining: 583ms
452:	learn: 0.

--- Tuned CatBoost RandomizedSearch 2---

In [10]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
param_dist_fine= {
    'depth': [4, 6, 8, 10, 12],
    'learning_rate': [0.001, 0.005, 0.01, 0.02, 0.03],
    'iterations': [300, 500, 700],
    'l2_leaf_reg': [1, 3, 5, 7, 9, 11],
    'border_count': [32, 64, 128],
    'bagging_temperature': [0.0, 0.2, 0.5, 1.0],
    'scale_pos_weight': [2, 5, 10, 20],
    'random_strength': [1, 2, 3, 5, 7],
    'subsample': [0.6, 0.8, 0.9, 1.0],
    'colsample_bylevel': [0.6, 0.8, 1.0]
}

rs_cat= RandomizedSearchCV(estimator=CatBoostClassifier(random_state=15, verbose=0), param_distributions= param_dist_fine, n_iter= 100, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2, random_state= 15)
rs_cat.fit(X_sm_tl, y_sm_tl)

tuned_cat2= rs_cat.best_estimator_

selector= RFE(estimator= tuned_cat2, n_features_to_select= 10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat2.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat2= tuned_cat2.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat2= tuned_cat2.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat2, labels= tuned_cat2.classes_)
cm
print(classification_report(y_test, y_pred_tuned_cat2, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat2, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat2))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat2))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      8517
           1       0.33      0.30      0.32       912

    accuracy                           0.87      9429
   macro avg       0.63      0.62      0.62      9429
weighted avg       0.87      0.87      0.87      9429

Precision: 0.33414634146341465
Accuracy: 0.8744299501537809
AUC: 0.7803603319676437


--- Visualisation ---

In [94]:
precision_scores= {
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision1:': precision_score(y_test, y_pred_tuned_grb1, zero_division= 0),
                    'Tuned GradientBoosting Precision2:': precision_score(y_test, y_pred_tuned_grb2, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'Tuned XGB Precision1:': precision_score(y_test, y_pred_tuned_xgb1, zero_division= 0),
                    'Tuned XGB Precision2:': precision_score(y_test, y_pred_tuned_xgb2, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'Tuned LGBM Precision1:': precision_score(y_test, y_pred_tuned_lgm1, zero_division= 0),
                    'Tuned LGBM Precision2:': precision_score(y_test, y_pred_tuned_lgm2, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0),
                    'Tuned CatBoost Precision1:': precision_score(y_test, y_pred_tuned_cat1, zero_division= 0),
                    'Tuned CatBoost Precision2:': precision_score(y_test, y_pred_tuned_cat2, zero_division= 0)
                  }

rfe_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
rfe_precision= rfe_precision.sort_values(by= 'Precision Score', ascending=False)
print(rfe_precision)

                                Model  Precision Score
11                    LGBM Precision:         0.509934
13                CatBoost Precision:         0.496350
14          Tuned CatBoost Precision:         0.493827
12              Tuned LGBM Precision:         0.486911
9                      XGB Precision:         0.457895
8   Tuned GradientBoosting Precision:         0.440433
6           Tuned AdaBoost Precision:         0.435262
10               Tuned XGB Precision:         0.421429
7         GradientBoosting Precision:         0.395230
4      Tuned Random Forest Precision:         0.365617
5                 AdaBoost Precision:         0.347743
2      Tuned Decision Tree Precision:         0.319672
3            Random Forest Precision:         0.317025
1            Decision Tree Precision:         0.261968
0      Logistic Regression Precision:         0.241969
                                Model  Precision Score
11                    LGBM Precision:         0.509934
13        

In [95]:
accuracy_scores= {
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'Tuned GradientBoosting Accuracy1:': accuracy_score(y_test, y_pred_tuned_grb1),
                    'Tuned GradientBoosting Accuracy2:': accuracy_score(y_test, y_pred_tuned_grb2),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'Tuned XGB Accuracy1:': accuracy_score(y_test, y_pred_tuned_xgb1),
                    'Tuned XGB Accuracy2:': accuracy_score(y_test, y_pred_tuned_xgb2),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'Tuned LGBM Accuracy1:': accuracy_score(y_test, y_pred_tuned_lgm1),
                    'Tuned LGBM Accuracy2:': accuracy_score(y_test, y_pred_tuned_lgm2),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat),
                    'Tuned CatBoost Accuracy1:': accuracy_score(y_test, y_pred_tuned_cat1),
                    'Tuned CatBoost Accuracy2:': accuracy_score(y_test, y_pred_tuned_cat2)
                  }

rfe_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
rfe_accuracy= rfe_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(rfe_accuracy)

                               Model  Accuracy Score
11                    LGBM Accuracy:        0.903595
13                CatBoost Accuracy:        0.903171
14          Tuned CatBoost Accuracy:        0.903065
12              Tuned LGBM Accuracy:        0.902747
9                      XGB Accuracy:        0.901580
8   Tuned GradientBoosting Accuracy:        0.899777
10               Tuned XGB Accuracy:        0.898611
6           Tuned AdaBoost Accuracy:        0.898293
4      Tuned Random Forest Accuracy:        0.891505
7         GradientBoosting Accuracy:        0.890232
2      Tuned Decision Tree Accuracy:        0.884611
3            Random Forest Accuracy:        0.883445
5                 AdaBoost Accuracy:        0.869657
1            Decision Tree Accuracy:        0.865309
0      Logistic Regression Accuracy:        0.743133
                               Model  Accuracy Score
11                    LGBM Accuracy:        0.903595
13                CatBoost Accuracy:        0.

In [96]:
auc_scores= {
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'Tuned GradientBoosting AUC1:': roc_auc_score(y_test, y_pred_prob_tuned_grb1),
                    'Tuned GradientBoosting AUC2:': roc_auc_score(y_test, y_pred_prob_tuned_grb2),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'Tuned XGB AUC1:': roc_auc_score(y_test, y_pred_prob_tuned_xgb1),
                    'Tuned XGB AUC2:': roc_auc_score(y_test, y_pred_prob_tuned_xgb2),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'Tuned LGBM AUC1:': roc_auc_score(y_test, y_pred_prob_tuned_lgm1),
                    'Tuned LGBM AUC2:': roc_auc_score(y_test, y_pred_prob_tuned_lgm2),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat),
                    'Tuned CatBoost AUC1:': roc_auc_score(y_test, y_pred_prob_tuned_cat1),
                    'Tuned CatBoost AUC2:': roc_auc_score(y_test, y_pred_prob_tuned_cat2)
                  }

rfe_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
rfe_auc= rfe_auc.sort_values(by= 'AUC Score', ascending=False)
print(rfe_auc)

                          Model  AUC Score
0      Logistic Regression AUC:   0.837566
5                 AdaBoost AUC:   0.824093
7         GradientBoosting AUC:   0.821224
8   Tuned GradientBoosting AUC:   0.819407
10               Tuned XGB AUC:   0.816064
11                    LGBM AUC:   0.808210
12              Tuned LGBM AUC:   0.807366
14          Tuned CatBoost AUC:   0.806100
9                      XGB AUC:   0.804016
6           Tuned AdaBoost AUC:   0.803449
13                CatBoost AUC:   0.802068
4      Tuned Random Forest AUC:   0.772369
3            Random Forest AUC:   0.753936
2      Tuned Decision Tree AUC:   0.732111
1            Decision Tree AUC:   0.592985
                          Model  AUC Score
0      Logistic Regression AUC:   0.837566
5                 AdaBoost AUC:   0.824093
7         GradientBoosting AUC:   0.821224
8   Tuned GradientBoosting AUC:   0.819407
10               Tuned XGB AUC:   0.816064
11                    LGBM AUC:   0.808210
12         